<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-21 13:09:33
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.94 L
Current:  1.27 C
-------------------
Today PnL: 45.10 K (0.36%)
Current PnL: -16.86 L (-11.95%)
CY Booked + Current PnL: -13.27 L (-9.41%)
-------------------
Total profit:  3.97 L
Total loss:  -20.83 L
-------------------
Total Booked + Current PnL: 13.71 L (12.03%)
Total Booked PnL: 30.57 L (26.81%)
Curr Year Booked PnL: 3.59 L (2.82%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.19 L (62.15%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (110.25%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ANGELONE,2328.67,3033.00,13.35,H-SC,91.15,102143.0,18311.0,7048.0,2.17,21.84,6.90,30.25,149.0,2.60,0.82,45.97,X40N,NTT,FINANCE
70,SBILIFE,1496.49,1924.99,-0.20,H-LC,73.96,148285.0,22580.0,13420.0,0.43,17.96,9.05,28.63,159.0,1.68,1.19,32.10,AR,ATH,INSURANCE
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,82.29,97859.0,19497.0,14532.0,-0.46,24.88,14.85,43.43,48.0,1.34,0.79,30.89,XY24,NTT,MISC
63,RELIANCE,1288.53,1526.00,4.61,X-LC,79.17,225197.0,21609.0,15921.0,0.05,10.61,7.07,18.43,161.0,1.36,1.81,22.67,XY25,NTT,REFINERIES
57,PIDILITIND,2504.06,3576.00,-19.11,X-LC,28.12,89700.0,14578.0,17581.0,-0.43,19.41,19.60,42.81,121.0,0.83,0.72,12.49,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.50,-24.57,H-SC,47.92,85770.0,-12210.0,34076.0,5.25,-12.46,39.73,22.32,102.0,-0.36,0.69,24.40,XR,ATH,FINANCE
21,COFFEEDAY,59.14,80.00,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
92,VIPIND,488.80,718.00,-874.01,H-SC,100.00,73759.0,-21557.0,66250.0,3.82,-22.62,89.82,46.89,173.0,-0.33,0.59,45.48,X40N,NTT,MISC
74,SIEMENS,4406.90,7969.85,-10.78,X-LC,87.50,80825.0,-29347.0,118425.0,3.37,-26.64,146.52,80.85,51.0,-0.25,0.65,19.04,AR,ATH,ELECTRICAL
13,BATAINDIA,1550.24,2096.00,-30.67,M-SC,56.25,104290.0,-24380.0,69676.0,3.34,-18.95,66.81,35.20,76.0,-0.35,0.84,6.40,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,REPCOHOME,515.07,880.0,-57.53,H-SC,16.67,157250.0,-47748.0,192993.0,-5.47,-23.29,122.73,70.85,113.0,-0.25,1.26,26.94,XY24,NTT,FINANCE
27,EASEMYTRIP,18.12,26.4,12.57,M-SC,2.08,108509.0,-68306.0,149102.0,-3.47,-38.63,137.41,45.70,22.0,-0.46,0.87,0.72,XY24,NTT,TRAVEL
44,JCHAC,2282.24,3542.0,19900.00,M-SC,18.75,97742.0,-30063.0,100606.0,-2.25,-23.52,102.93,55.20,210.0,-0.30,0.79,42.68,X40N,BTT,AC
41,INDUSINDBK,1354.50,1800.0,-41.30,H-MC,4.17,43616.0,-33590.0,58982.0,-2.18,-43.51,135.23,32.89,44.0,-0.57,0.35,20.12,XR,NTT,BANKS
14,BERGEPAINT,558.54,680.0,-19.11,H-MC,26.04,173494.0,-3563.0,42072.0,-1.76,-2.01,24.25,21.75,183.0,-0.08,1.39,24.10,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.28,H-SC,58.85,105305.0,3578.0,110791.0,1.26,3.52,105.21,112.43,1.0,0.03,0.85,24.29,AR,ATH,MISC
83,TCS,3928.29,4998.00,-15.52,X-LC,58.85,190350.0,-21778.0,79547.0,0.76,-10.27,41.79,27.23,3.0,-0.27,1.53,9.08,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-52.03,M-LC,89.58,191661.0,-13984.0,90560.0,0.60,-6.80,47.25,37.24,4.0,-0.15,1.54,24.75,XY24,NTT,AUTO
10,AWL,342.88,485.00,-69.39,H-MC,3.12,165672.0,-52057.0,142296.0,-0.55,-23.91,85.89,41.45,4.0,-0.37,1.33,10.67,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-12.42,X-LC,71.88,217744.0,18583.0,98486.0,0.38,9.33,45.23,58.78,6.0,0.19,1.75,13.68,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ANGELONE,2328.67,3033.00,13.35,H-SC,91.15,102143.0,18311.0,7048.0,2.17,21.84,6.90,30.25,149.0,2.60,0.82,45.97,X40N,NTT,FINANCE
58,POLYCAB,5005.95,7541.35,7.93,H-MC,84.38,102570.0,17469.0,25632.0,0.33,20.53,24.99,50.65,56.0,0.68,0.82,28.93,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,82.29,97859.0,19497.0,14532.0,-0.46,24.88,14.85,43.43,48.0,1.34,0.79,30.89,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,43.16,M-SC,67.71,148304.0,10726.0,83688.0,-0.28,7.80,56.43,68.62,66.0,0.13,1.19,28.57,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,13.35,H-SC,91.15,102143.0,18311.0,7048.0,2.17,21.84,6.90,30.25,149.0,2.60,0.82,45.97,X40N,NTT,FINANCE
58,POLYCAB,5005.95,7541.35,7.93,H-MC,84.38,102570.0,17469.0,25632.0,0.33,20.53,24.99,50.65,56.0,0.68,0.82,28.93,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,1.78,H-SC,85.42,150178.0,12318.0,25440.0,1.50,8.93,16.94,27.39,64.0,0.48,1.21,31.46,X40,ATH,FMCG
56,PFIZER,4236.33,6318.30,-2.52,H-SC,96.88,152012.0,20686.0,43855.0,-0.93,15.75,28.85,49.15,94.0,0.47,1.22,25.29,X40,ATH,PHARMA


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-3.11,H-SC,19.79,109038.0,7034.0,37989.0,0.70,6.90,34.84,44.14,31.0,0.19,0.88,24.26,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-31.87,M-MC,20.83,133294.0,4142.0,82922.0,0.83,3.21,62.21,67.42,57.0,0.05,1.07,29.64,XR,NTT,BANKS
95,WIPRO,243.46,420.00,-9.26,M-LC,57.29,155496.0,4551.0,104898.0,0.44,3.01,67.46,72.51,49.0,0.04,1.25,17.08,XR,NTT,IT
39,INDIAMART,2311.97,4911.36,-51.28,H-SC,58.85,105305.0,3578.0,110791.0,1.26,3.52,105.21,112.43,1.0,0.03,0.85,24.29,AR,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-2.86,H-MC,72.92,148382.0,-2531.0,62988.0,-0.05,-1.68,42.45,40.06,27.0,-0.04,1.19,21.37,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.0,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
53,MASFIN,326.60,399.5,-24.57,H-SC,47.92,85770.0,-12210.0,34076.0,5.25,-12.46,39.73,22.32,102.0,-0.36,0.69,24.40,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-2.58,M-LC,82.29,97859.0,19497.0,14532.0,-0.46,24.88,14.85,43.43,48.0,1.34,0.79,30.89,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.0,13.35,H-SC,91.15,102143.0,18311.0,7048.0,2.17,21.84,6.90,30.25,149.0,2.60,0.82,45.97,X40N,NTT,FINANCE
90,VALIANTORG,512.64,838.0,-345.79,H-SC,77.08,104507.0,-28779.0,113369.0,1.18,-21.59,108.48,63.47,145.0,-0.25,0.84,75.18,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-874.01,H-SC,100.00,73759.0,-21557.0,66250.0,3.82,-22.62,89.82,46.89,173.0,-0.33,0.59,45.48,X40N,NTT,MISC
21,COFFEEDAY,59.14,80.00,-55.73,L-SC,98.96,64646.0,-48903.0,88953.0,4.99,-43.07,137.60,35.27,181.0,-0.55,0.52,58.22,XR,NTT,HOTELS
51,LTIM,5664.39,7262.39,-5.69,X-LC,97.92,151110.0,-18822.0,66760.0,0.43,-11.08,44.18,28.21,32.0,-0.28,1.21,24.15,X200,ATH,IT
56,PFIZER,4236.33,6318.30,-2.52,H-SC,96.88,152012.0,20686.0,43855.0,-0.93,15.75,28.85,49.15,94.0,0.47,1.22,25.29,X40,ATH,PHARMA
94,WHIRLPOOL,1151.42,2270.00,-35.67,M-SC,95.83,161912.0,15682.0,126372.0,-1.59,10.72,78.05,97.15,28.0,0.12,1.30,38.32,X40,NTT,DURABLES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.34
1,25,38.57
2,50,67.74


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.21
X40N    14.37
XY25    14.09
X40     13.53
AR      11.43
XR      10.04
X200     1.21
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.44
M-SC    17.89
X-LC    16.30
H-MC     8.43
M-LC     7.69
H-LC     6.06
M-MC     5.66
X-MC     5.64
X-SC     2.85
L-SC     2.36
L-LC     1.20
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.79,-1.10,33.27
BANKS,11.05,-9.59,63.35
IT,9.23,-20.51,87.74
FINANCE,6.87,-21.40,77.36
MISC,6.52,-30.68,86.87
INSURANCE,6.23,0.46,29.55
PAINTS,5.56,-18.59,53.68
AUTO,4.38,-11.05,59.46
ELECTRICAL,4.20,-6.38,60.59


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2988735.0,27
X40N,1205068.0,18
AR,1171725.0,13
XR,1100025.0,14
X40,719880.0,11
XY25,460555.0,10
SR,206059.0,2
X200,66760.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2308509.0,22
M-SC,2013796.0,21
X-LC,711191.0,12
M-MC,563869.0,5
H-MC,474100.0,8
X-SC,353217.0,4
M-LC,319051.0,6
L-SC,304535.0,4
X-MC,304312.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      913524.0      6
M-SC       XY24      761103.0      6
H-SC       AR        536436.0      3
           X40N      513772.0      7
M-SC       X40N      449933.0      6
M-MC       XY24      396835.0      3
X-LC       X40       385365.0      5
M-SC       XR        304886.0      4
H-SC       XR        275482.0      4
M-SC       X40       196048.0      2
M-LC       XY24      193211.0      4
H-LC       AR        188210.0      4
H-MC       XY24      184368.0      2
L-SC       XR        154155.0      2
X-MC       XY24      144544.0      2
X-LC       AR        142370.0      2
M-SC       XY25      123252.0      1
X-SC       SR        120743.0      1
           XR        118700.0      1
X-LC       XY25      116696.0      4
X-SC       X40N      113774.0      2
M-LC       XR        104898.0      1
X-MC       X40N      101957.0      2
M-SC       AR         93258.0      1
L-SC       XY24       86029.0      1
M-SC       SR         85316.0      1
M-MC       AR         84112.0      1
           XR         82922.0      1
H-MC       XY25       72958.0      1
H-SC       X40        69295.0      2
H-MC       X40        69172.0      2
X-LC       X200       66760.0      1
L-SC       AR         64351.0      1
H-MC       AR         62988.0      1
           XR         58982.0      1
X-MC       XY25       57811.0      1
L-LC       XY25       39695.0      1
H-LC       XY25       29201.0      1
H-MC       X40N       25632.0      1
M-LC       XY25       20942.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
